In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn import tree
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import tree
from scipy import stats

In [2]:
letterp2_data = pd.read_csv('letter-recognition.data', header=None)
letterp2_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [3]:
letterp2_data.columns = ['lettr capital letter', 'x-box horizontal position of box', 'y-box vertical position of box', 'width width of box', 'high height of box', 'onpix total # on pixels', 'x-bar mean x of on pixels in box', 'y-bar mean y of on pixels in box', 'x2bar mean x variance', 'y2bar mean y variance', 'xybar mean x y correlation', 'x2ybr mean of x * x * y', 'xy2br mean of x * y * y', 'x-ege mean edge count left to right', ' xegvy correlation of x-ege with y', 'y-ege mean edge count bottom to top', 'yegvx correlation of y-ege with x']
letterp2_data = letterp2_data.dropna()
letterp2_data[:10]

,lettr capital letter,x-box horizontal position of box,y-box vertical position of box,width width of box,high height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
5,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
6,B,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
7,A,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
8,J,2,2,4,4,2,10,6,2,6,12,4,8,1,6,1,7
9,M,11,15,13,9,7,13,2,6,2,12,1,9,8,1,1,8


In [4]:
def summerize_data(df):
    for column in df.columns:
        print(column)
        if df.dtypes[column] == np.object:
            print(df[column].value_counts())
        else:
            print(df[column].describe())
        print('\n')

In [5]:
summerize_data(letterp2_data)

lettr capital letter
U    813
D    805
P    803
T    796
M    792
A    789
X    787
Y    786
Q    783
N    783
F    775
G    773
E    768
B    766
V    764
L    761
R    758
I    755
O    753
W    752
S    748
J    747
K    739
C    736
Z    734
H    734
Name: lettr capital letter, dtype: int64


x-box horizontal position of box
count    20000.000000
mean         4.023550
std          1.913212
min          0.000000
25%          3.000000
50%          4.000000
75%          5.000000
max         15.000000
Name: x-box horizontal position of box, dtype: float64


y-box vertical position of box
count    20000.000000
mean         7.035500
std          3.304555
min          0.000000
25%          5.000000
50%          7.000000
75%          9.000000
max         15.000000
Name: y-box vertical position of box, dtype: float64


width width of box
count    20000.000000
mean         5.121850
std          2.014573
min          0.000000
25%          4.000000
50%          5.000000
75%          6.000000
m

In [6]:
letterp2_data = letterp2_data.replace({'lettr capital letter': {'A':1, 'B':1, 'C':1, 'D':1, 'E':1, 'F':1, 'G':1, 'H':1, 'I':1, 'J':1, 'K':1, 'L':1, 'M':1}})
letterp2_data = letterp2_data.replace({'lettr capital letter': {'N':0, 'O':0, 'P':0, 'Q':0, 'R':0, 'S':0, 'T':0, 'U':0, 'V':0, 'W':0, 'X':0, 'Y':0, 'Z':0}})
letterp2_data[:10]

,lettr capital letter,x-box horizontal position of box,y-box vertical position of box,width width of box,high height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,1,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,0,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,1,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
5,0,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
6,1,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
7,1,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
8,1,2,2,4,4,2,10,6,2,6,12,4,8,1,6,1,7
9,1,11,15,13,9,7,13,2,6,2,12,1,9,8,1,1,8


In [7]:
letterp2_data = letterp2_data.dropna()
letterp2_data

,lettr capital letter,x-box horizontal position of box,y-box vertical position of box,width width of box,high height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,1,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,0,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,1,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,1,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,0,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,0,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [8]:
summerize_data(letterp2_data)

lettr capital letter
count    20000.000000
mean         0.497000
std          0.500004
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: lettr capital letter, dtype: float64


x-box horizontal position of box
count    20000.000000
mean         4.023550
std          1.913212
min          0.000000
25%          3.000000
50%          4.000000
75%          5.000000
max         15.000000
Name: x-box horizontal position of box, dtype: float64


y-box vertical position of box
count    20000.000000
mean         7.035500
std          3.304555
min          0.000000
25%          5.000000
50%          7.000000
75%          9.000000
max         15.000000
Name: y-box vertical position of box, dtype: float64


width width of box
count    20000.000000
mean         5.121850
std          2.014573
min          0.000000
25%          4.000000
50%          5.000000
75%          6.000000
max         15.000000
Name: width width of box, dtype: fl

In [9]:
letterp2_2 = letterp2_data.drop(['lettr capital letter'], axis=1)
letterp2_2

,x-box horizontal position of box,y-box vertical position of box,width width of box,high height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [10]:
letterp2_data['lettr capital letter']

0        0
1        1
2        1
3        0
4        1
        ..
19995    1
19996    1
19997    0
19998    0
19999    1
Name: lettr capital letter, Length: 20000, dtype: int64

In [49]:
#Random Forest
for i in range(5):
    print('trial', i+1)
    letterp2_rf_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(letterp2_2, letterp2_data['lettr capital letter'], train_size=5000)
        X_train = pd.DataFrame(X_train, columns=X_train.columns)
        X_test = X_test

        for k in range(5):
            letterp2_rf = RandomForestClassifier(criterion='entropy')
            param_grid = [{'n_estimators': [1024], 'max_features': [1, 2, 4, 6, 8, 12, 16]}]
            #n_list = [1, 2 , 4 , 6, 8, 12, 16, 20]
            letterp2_rf_grid = GridSearchCV(letterp2_rf, param_grid, cv=KFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            letterp2_rf_grid.fit(X_train, y_train)
            print('mean test accuracy:', letterp2_rf_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', letterp2_rf_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', letterp2_rf_grid.cv_results_['mean_test_f1_micro'])
            letterp2_rf_results = pd.DataFrame(letterp2_rf_grid.cv_results_['params'])
            letterp2_rf_results['score_acc'] = letterp2_rf_grid.cv_results_['mean_test_accuracy']
            letterp2_rf_list.append(letterp2_rf_grid.cv_results_['mean_test_accuracy'])
            letterp2_rf_cols = letterp2_rf_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            letterp2_rf_results.columns = letterp2_rf_cols
            letterp2_rf_results = letterp2_rf_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(letterp2_rf_results)

trial 1
mean test accuracy: [0.9334 0.9366 0.9384 0.9358 0.9364 0.9322 0.9298]
mean test roc auo ovr: [0.98540959 0.98732296 0.98792413 0.98771018 0.9873727  0.98670883
 0.98514889]
mean test f1 micro: [0.9334 0.9366 0.9384 0.9358 0.9364 0.9322 0.9298]
   max_features  n_estimators  score_acc
0             4          1024     0.9384
1             2          1024     0.9366
2             8          1024     0.9364
3             6          1024     0.9358
4             1          1024     0.9334
5            12          1024     0.9322
6            16          1024     0.9298
mean test accuracy: [0.9316 0.9388 0.9392 0.9366 0.9356 0.9332 0.9288]
mean test roc auo ovr: [0.9852695  0.98737793 0.98776269 0.98756653 0.98759606 0.98663577
 0.98516291]
mean test f1 micro: [0.9316 0.9388 0.9392 0.9366 0.9356 0.9332 0.9288]
   max_features  n_estimators  score_acc
0             4          1024     0.9392
1             2          1024     0.9388
2             6          1024     0.9366
3         

mean test accuracy: [0.9354 0.9394 0.9394 0.9402 0.939  0.9386 0.937 ]
mean test roc auo ovr: [0.98598989 0.98793869 0.98781242 0.98795012 0.98751153 0.98675976
 0.98537513]
mean test f1 micro: [0.9354 0.9394 0.9394 0.9402 0.939  0.9386 0.937 ]
   max_features  n_estimators  score_acc
0             6          1024     0.9402
1             2          1024     0.9394
2             4          1024     0.9394
3             8          1024     0.9390
4            12          1024     0.9386
5            16          1024     0.9370
6             1          1024     0.9354
mean test accuracy: [0.9342 0.9398 0.9402 0.9396 0.9404 0.9372 0.9362]
mean test roc auo ovr: [0.98594781 0.98796552 0.98810139 0.98796843 0.98771407 0.98668232
 0.98526375]
mean test f1 micro: [0.9342 0.9398 0.9402 0.9396 0.9404 0.9372 0.9362]
   max_features  n_estimators  score_acc
0             8          1024     0.9404
1             4          1024     0.9402
2             2          1024     0.9398
3             6   

mean test accuracy: [0.9304 0.9362 0.9388 0.9386 0.9384 0.9382 0.9382]
mean test roc auo ovr: [0.98683105 0.98866342 0.98881592 0.98871901 0.98871125 0.98825641
 0.98740293]
mean test f1 micro: [0.9304 0.9362 0.9388 0.9386 0.9384 0.9382 0.9382]
   max_features  n_estimators  score_acc
0             4          1024     0.9388
1             6          1024     0.9386
2             8          1024     0.9384
3            12          1024     0.9382
4            16          1024     0.9382
5             2          1024     0.9362
6             1          1024     0.9304
mean test accuracy: [0.9322 0.938  0.938  0.939  0.939  0.9368 0.9382]
mean test roc auo ovr: [0.9867285  0.98863111 0.98892042 0.98862766 0.98865627 0.9882258
 0.98733254]
mean test f1 micro: [0.9322 0.938  0.938  0.939  0.939  0.9368 0.9382]
   max_features  n_estimators  score_acc
0             6          1024     0.9390
1             8          1024     0.9390
2            16          1024     0.9382
3             2    

mean test accuracy: [0.9328 0.9392 0.94   0.9376 0.9394 0.9388 0.9376]
mean test roc auo ovr: [0.98637545 0.98824169 0.98888697 0.98871928 0.98867273 0.9881428
 0.98661297]
mean test f1 micro: [0.9328 0.9392 0.94   0.9376 0.9394 0.9388 0.9376]
   max_features  n_estimators  score_acc
0             4          1024     0.9400
1             8          1024     0.9394
2             2          1024     0.9392
3            12          1024     0.9388
4             6          1024     0.9376
5            16          1024     0.9376
6             1          1024     0.9328
mean test accuracy: [0.9324 0.9384 0.9398 0.9388 0.9388 0.9368 0.937 ]
mean test roc auo ovr: [0.98600834 0.98831418 0.98885301 0.98869323 0.98860603 0.98785412
 0.98674229]
mean test f1 micro: [0.9324 0.9384 0.9398 0.9388 0.9388 0.9368 0.937 ]
   max_features  n_estimators  score_acc
0             4          1024     0.9398
1             6          1024     0.9388
2             8          1024     0.9388
3             2    

mean test accuracy: [0.9318 0.939  0.9404 0.9404 0.9392 0.9382 0.9366]
mean test roc auo ovr: [0.98632564 0.98812786 0.98838347 0.9883265  0.98801545 0.98738143
 0.98664096]
mean test f1 micro: [0.9318 0.939  0.9404 0.9404 0.9392 0.9382 0.9366]
   max_features  n_estimators  score_acc
0             4          1024     0.9404
1             6          1024     0.9404
2             8          1024     0.9392
3             2          1024     0.9390
4            12          1024     0.9382
5            16          1024     0.9366
6             1          1024     0.9318
mean test accuracy: [0.9324 0.939  0.9388 0.9394 0.9408 0.938  0.9378]
mean test roc auo ovr: [0.98653042 0.98825774 0.98829504 0.98820594 0.9881352  0.98731042
 0.98642015]
mean test f1 micro: [0.9324 0.939  0.9388 0.9394 0.9408 0.938  0.9378]
   max_features  n_estimators  score_acc
0             8          1024     0.9408
1             6          1024     0.9394
2             2          1024     0.9390
3             4   

mean test accuracy: [0.9376 0.9404 0.9434 0.9414 0.942  0.9414 0.9382]
mean test roc auo ovr: [0.98699635 0.98843426 0.98825027 0.98782726 0.98762914 0.98666194
 0.9853921 ]
mean test f1 micro: [0.9376 0.9404 0.9434 0.9414 0.942  0.9414 0.9382]
   max_features  n_estimators  score_acc
0             4          1024     0.9434
1             8          1024     0.9420
2             6          1024     0.9414
3            12          1024     0.9414
4             2          1024     0.9404
5            16          1024     0.9382
6             1          1024     0.9376
mean test accuracy: [0.9362 0.941  0.9428 0.9416 0.9422 0.9422 0.9394]
mean test roc auo ovr: [0.98685108 0.98857835 0.9883219  0.98782168 0.98723536 0.98671556
 0.98555064]
mean test f1 micro: [0.9362 0.941  0.9428 0.9416 0.9422 0.9422 0.9394]
   max_features  n_estimators  score_acc
0             4          1024     0.9428
1             8          1024     0.9422
2            12          1024     0.9422
3             6   

mean test accuracy: [0.9312 0.9404 0.9382 0.9392 0.9414 0.9378 0.9366]
mean test roc auo ovr: [0.98597269 0.98769407 0.98810755 0.9880823  0.98774266 0.98699857
 0.98576356]
mean test f1 micro: [0.9312 0.9404 0.9382 0.9392 0.9414 0.9378 0.9366]
   max_features  n_estimators  score_acc
0             8          1024     0.9414
1             2          1024     0.9404
2             6          1024     0.9392
3             4          1024     0.9382
4            12          1024     0.9378
5            16          1024     0.9366
6             1          1024     0.9312
mean test accuracy: [0.9332 0.9398 0.9404 0.9384 0.9402 0.9386 0.937 ]
mean test roc auo ovr: [0.98596707 0.98774246 0.98803666 0.98796706 0.987625   0.98692421
 0.98587172]
mean test f1 micro: [0.9332 0.9398 0.9404 0.9384 0.9402 0.9386 0.937 ]
   max_features  n_estimators  score_acc
0             4          1024     0.9404
1             8          1024     0.9402
2             2          1024     0.9398
3            12   

mean test accuracy: [0.9304 0.9384 0.9422 0.9416 0.9426 0.9396 0.9314]
mean test roc auo ovr: [0.9858159  0.9882015  0.98874436 0.98823869 0.98815513 0.98702576
 0.98468065]
mean test f1 micro: [0.9304 0.9384 0.9422 0.9416 0.9426 0.9396 0.9314]
   max_features  n_estimators  score_acc
0             8          1024     0.9426
1             4          1024     0.9422
2             6          1024     0.9416
3            12          1024     0.9396
4             2          1024     0.9384
5            16          1024     0.9314
6             1          1024     0.9304
mean test accuracy: [0.9312 0.9384 0.9422 0.943  0.9402 0.9378 0.9322]
mean test roc auo ovr: [0.98590556 0.98808908 0.98861199 0.98837553 0.98798672 0.98702698
 0.98455664]
mean test f1 micro: [0.9312 0.9384 0.9422 0.943  0.9402 0.9378 0.9322]
   max_features  n_estimators  score_acc
0             6          1024     0.9430
1             4          1024     0.9422
2             8          1024     0.9402
3             2   

mean test accuracy: [0.9378 0.9384 0.9376 0.9382 0.9392 0.936  0.9322]
mean test roc auo ovr: [0.98715628 0.98848821 0.98848626 0.98775219 0.98746298 0.98640197
 0.98432886]
mean test f1 micro: [0.9378 0.9384 0.9376 0.9382 0.9392 0.936  0.9322]
   max_features  n_estimators  score_acc
0             8          1024     0.9392
1             2          1024     0.9384
2             6          1024     0.9382
3             1          1024     0.9378
4             4          1024     0.9376
5            12          1024     0.9360
6            16          1024     0.9322
mean test accuracy: [0.936  0.9408 0.939  0.9392 0.9386 0.9352 0.9304]
mean test roc auo ovr: [0.98746515 0.98865143 0.98845829 0.98790098 0.98716808 0.98628199
 0.98403442]
mean test f1 micro: [0.936  0.9408 0.939  0.9392 0.9386 0.9352 0.9304]
   max_features  n_estimators  score_acc
0             2          1024     0.9408
1             6          1024     0.9392
2             4          1024     0.9390
3             8   

In [50]:
letterp2_rf_list

[array([0.9396, 0.9458, 0.9444, 0.9466, 0.9456, 0.9446, 0.9414]),
 array([0.939 , 0.9442, 0.9464, 0.9462, 0.9458, 0.9426, 0.9414]),
 array([0.9392, 0.9446, 0.946 , 0.946 , 0.9448, 0.9444, 0.9416]),
 array([0.9384, 0.9454, 0.9452, 0.9452, 0.9456, 0.9432, 0.9436]),
 array([0.9412, 0.9452, 0.9464, 0.9464, 0.945 , 0.9446, 0.9422]),
 array([0.9304, 0.9384, 0.9422, 0.9416, 0.9426, 0.9396, 0.9314]),
 array([0.9312, 0.9384, 0.9422, 0.943 , 0.9402, 0.9378, 0.9322]),
 array([0.9312, 0.9382, 0.9402, 0.9426, 0.94  , 0.9396, 0.9342]),
 array([0.9322, 0.937 , 0.94  , 0.943 , 0.9428, 0.9392, 0.933 ]),
 array([0.9292, 0.9388, 0.9428, 0.9402, 0.9422, 0.9396, 0.9336]),
 array([0.9372, 0.9416, 0.9434, 0.9434, 0.9438, 0.9428, 0.9416]),
 array([0.936 , 0.94  , 0.9442, 0.9428, 0.9444, 0.9422, 0.9418]),
 array([0.9362, 0.9436, 0.9424, 0.944 , 0.9436, 0.9424, 0.942 ]),
 array([0.9374, 0.9424, 0.9434, 0.943 , 0.9446, 0.9432, 0.9414]),
 array([0.9376, 0.9422, 0.9436, 0.9436, 0.9442, 0.9414, 0.9406]),
 array([0.

In [51]:
letterp2_rf_grid.cv_results_

{'mean_fit_time': array([ 2.77511582,  3.23829374,  4.20285945,  5.21063719,  6.19181523,
         8.16574345, 10.12767978]),
 'std_fit_time': array([0.00936513, 0.00877921, 0.01942484, 0.0244323 , 0.03437074,
        0.05627626, 0.08026963]),
 'mean_score_time': array([0.47618742, 0.44178405, 0.42719035, 0.40679002, 0.40098538,
        0.39618382, 0.39038968]),
 'std_score_time': array([0.00240484, 0.00193983, 0.01844629, 0.00172136, 0.00126038,
        0.00470328, 0.00215613]),
 'param_max_features': masked_array(data=[1, 2, 4, 6, 8, 12, 16],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1024, 1024, 1024, 1024, 1024, 1024, 1024],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_features': 1, 'n_estimators': 1024},
  {'max_features': 2, 'n_estimators': 1024},
  {'max_features': 4

In [52]:
letterp2_rf_grid.cv_results_['rank_test_accuracy']

array([6, 4, 1, 3, 1, 5, 7])

In [53]:
letterp2_rf_grid.cv_results_['params'][ np.argmin(letterp2_rf_grid.cv_results_['rank_test_accuracy'])]

{'max_features': 4, 'n_estimators': 1024}

In [54]:
letterp2_rf_grid.cv_results_['params'][ np.argmin(letterp2_rf_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'max_features': 4, 'n_estimators': 1024}

In [55]:
letterp2_rf_results = pd.DataFrame(letterp2_rf_grid.cv_results_['params'])
letterp2_rf_results['score_acc'] = letterp2_rf_grid.cv_results_['mean_test_accuracy']
letterp2_rf_cols = letterp2_rf_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
letterp2_rf_results.columns = letterp2_rf_cols
letterp2_rf_results = letterp2_rf_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
letterp2_rf_results

,max_features,n_estimators,score_acc
0,4,1024,0.9394
1,8,1024,0.9394
2,6,1024,0.9384
3,2,1024,0.9382
4,12,1024,0.9374
5,1,1024,0.9358
6,16,1024,0.9314


In [56]:
X_train, X_test, y_train, y_test = train_test_split(letterp2_2, letterp2_data['lettr capital letter'], train_size=5000)
X_train = pd.DataFrame(X_train, columns=X_train.columns)
letterp2_rf_best = RandomForestClassifier(n_estimators=1024, criterion='entropy', max_features=4)
letterp2_rf_best.fit(X_train, y_train)
letterp2_pred_rf = letterp2_rf_best.predict(X_test)
print('Random Forest:', classification_report(y_test, letterp2_pred_rf))

Random Forest:               precision    recall  f1-score   support

           0       0.94      0.95      0.95      7557
           1       0.95      0.94      0.94      7443

    accuracy                           0.95     15000
   macro avg       0.95      0.95      0.95     15000
weighted avg       0.95      0.95      0.95     15000



In [41]:
#Logistic Regression
for i in range(5):
    print('trial', i+1)
    letterp2_logreg_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(letterp2_2, letterp2_data['lettr capital letter'], train_size=5000)
        sc = StandardScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = sc.transform(X_test)

        for k in range(5):
            pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', linear_model.LogisticRegression())])
            search_space = [{'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']}]
            letterp2_logreg_grid = GridSearchCV(pipe, search_space, cv=KFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            letterp2_logreg_grid.fit(X_train, y_train)
            print('mean test accuracy:', letterp2_logreg_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', letterp2_logreg_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', letterp2_logreg_grid.cv_results_['mean_test_f1_micro'])
            letterp2_logreg_results = pd.DataFrame(letterp2_logreg_grid.cv_results_['params'])
            letterp2_logreg_results['score_acc'] = letterp2_logreg_grid.cv_results_['mean_test_accuracy']
            letterp2_logreg_list.append(letterp2_logreg_grid.cv_results_['mean_test_accuracy'])
            letterp2_logreg_cols = letterp2_logreg_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            letterp2_logreg_results.columns = letterp2_logreg_cols
            letterp2_logreg_results = letterp2_logreg_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(letterp2_logreg_results)

trial 1
mean test accuracy: [0.4956 0.6698 0.4884 0.6846 0.6898 0.7076 0.7104 0.7098 0.7104 0.7108
 0.7108 0.7108 0.7106 0.7106 0.7106 0.7106 0.7106 0.7104 0.6698 0.6846
 0.7076 0.7098 0.7106 0.7108 0.7108 0.7106 0.7106 0.7106 0.7106]
mean test roc auo ovr: [0.5        0.7559663  0.5        0.77792633 0.77739301 0.79936297
 0.80287238 0.80405459 0.80426024 0.80437082 0.80437318 0.80438437
 0.80437957 0.80438678 0.80438518 0.80438197 0.80438198 0.80438518
 0.75596631 0.77792473 0.79936457 0.80405298 0.80436841 0.80438597
 0.80438277 0.80438517 0.80438437 0.80438437 0.80438277]
mean test f1 micro: [0.4956 0.6698 0.4884 0.6846 0.6898 0.7076 0.7104 0.7098 0.7104 0.7108
 0.7108 0.7108 0.7106 0.7106 0.7106 0.7106 0.7106 0.7104 0.6698 0.6846
 0.7076 0.7098 0.7106 0.7108 0.7108 0.7106 0.7106 0.7106 0.7106]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)    100.0000      l2  lbfgs     0.7108
1   LogisticRegression(max_iter=5000) 

mean test accuracy: [0.4884 0.6698 0.4904 0.6846 0.6898 0.7076 0.7104 0.7098 0.7104 0.7106
 0.7108 0.7108 0.7106 0.7106 0.7106 0.7106 0.7106 0.7106 0.6698 0.6846
 0.7076 0.7098 0.7106 0.7108 0.7108 0.7106 0.7106 0.7106 0.7106]
mean test roc auo ovr: [0.5        0.7559663  0.5        0.77792392 0.77739701 0.79936217
 0.80287238 0.80404818 0.80425944 0.80437082 0.80436997 0.80438757
 0.80438437 0.80438277 0.80438118 0.80438438 0.80438038 0.80438518
 0.75596631 0.77792473 0.79936457 0.80405298 0.80436841 0.80438597
 0.80438277 0.80438517 0.80438437 0.80438437 0.80438438]
mean test f1 micro: [0.4884 0.6698 0.4904 0.6846 0.6898 0.7076 0.7104 0.7098 0.7104 0.7106
 0.7108 0.7108 0.7106 0.7106 0.7106 0.7106 0.7106 0.7106 0.6698 0.6846
 0.7076 0.7098 0.7106 0.7108 0.7108 0.7106 0.7106 0.7106 0.7106]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)    100.0000      l2  lbfgs     0.7108
1   LogisticRegression(max_iter=5000)     10.0

mean test accuracy: [0.499  0.6708 0.5026 0.6876 0.6898 0.7188 0.723  0.7246 0.724  0.7246
 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.6708 0.6876
 0.7188 0.7248 0.7246 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248]
mean test roc auo ovr: [0.5        0.76054102 0.5        0.78217848 0.77910037 0.80381194
 0.80762278 0.80864296 0.80891097 0.80892776 0.80893976 0.80893416
 0.80893655 0.80894376 0.80894136 0.80894136 0.80893656 0.80893575
 0.76053862 0.78218008 0.80381114 0.80864376 0.80892456 0.80893175
 0.80893736 0.80893896 0.80893896 0.80893896 0.80893816]
mean test f1 micro: [0.499  0.6708 0.5026 0.6876 0.6898 0.7188 0.723  0.7246 0.724  0.7246
 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.6708 0.6876
 0.7188 0.7248 0.7246 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)   1000.0000      l1   saga     0.7248
1   LogisticRegression(max_iter=5000)    100.0

mean test accuracy: [0.5034 0.6708 0.4914 0.6876 0.6898 0.7188 0.723  0.7248 0.724  0.7246
 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.6708 0.6876
 0.7188 0.7248 0.7246 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248]
mean test roc auo ovr: [0.5        0.76053702 0.5        0.78218008 0.77910197 0.80381034
 0.80762198 0.80864536 0.80890057 0.80892776 0.80894136 0.80893095
 0.80893575 0.80893735 0.80894056 0.80893896 0.80893576 0.80893896
 0.76053862 0.78218008 0.80381114 0.80864376 0.80892456 0.80893175
 0.80893736 0.80893896 0.80893896 0.80893896 0.80893896]
mean test f1 micro: [0.5034 0.6708 0.4914 0.6876 0.6898 0.7188 0.723  0.7248 0.724  0.7246
 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248 0.6708 0.6876
 0.7188 0.7248 0.7246 0.7248 0.7248 0.7248 0.7248 0.7248 0.7248]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)   1000.0000      l1   saga     0.7248
1   LogisticRegression(max_iter=5000)    100.0

mean test accuracy: [0.4982 0.6848 0.501  0.7012 0.6996 0.7236 0.7324 0.734  0.7338 0.734
 0.7346 0.7346 0.7348 0.7348 0.7348 0.7348 0.7348 0.7348 0.6848 0.7014
 0.7236 0.734  0.734  0.7346 0.7348 0.7348 0.7348 0.7348 0.7348]
mean test roc auo ovr: [0.5        0.76163737 0.5        0.78586577 0.78638999 0.81050238
 0.81555902 0.81652352 0.81685891 0.81691973 0.81692933 0.81693653
 0.81694614 0.81694534 0.81694774 0.81694374 0.81694054 0.81694454
 0.76163737 0.78586738 0.81049838 0.81652272 0.81691973 0.81693013
 0.81694774 0.81694614 0.81694534 0.81694534 0.81693974]
mean test f1 micro: [0.4982 0.6848 0.501  0.7012 0.6996 0.7236 0.7324 0.734  0.7338 0.734
 0.7346 0.7346 0.7348 0.7348 0.7348 0.7348 0.7348 0.7348 0.6848 0.7014
 0.7236 0.734  0.734  0.7346 0.7348 0.7348 0.7348 0.7348 0.7348]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)   1000.0000      l1   saga     0.7348
1   LogisticRegression(max_iter=5000)    100.000

...

mean test accuracy: [0.5064 0.676  0.5072 0.6994 0.7008 0.7218 0.7274 0.726  0.7268 0.727
 0.7266 0.7266 0.7266 0.7266 0.7264 0.7264 0.7264 0.7264 0.6762 0.6994
 0.7218 0.726  0.727  0.7266 0.7264 0.7264 0.7264 0.7264 0.7266]
mean test roc auo ovr: [0.5        0.76828881 0.5        0.79064971 0.79104116 0.81185763
 0.81584831 0.81682721 0.81720256 0.81725938 0.81730102 0.81731142
 0.81732102 0.81732262 0.81732262 0.81732422 0.81732182 0.81732502
 0.76828881 0.7906465  0.81185523 0.81682961 0.81725938 0.81730662
 0.81732182 0.81732502 0.81732502 0.81732502 0.81732982]
mean test f1 micro: [0.5064 0.676  0.5072 0.6994 0.7008 0.7218 0.7274 0.726  0.7268 0.727
 0.7266 0.7266 0.7266 0.7266 0.7264 0.7264 0.7264 0.7264 0.6762 0.6994
 0.7218 0.726  0.727  0.7266 0.7264 0.7264 0.7264 0.7264 0.7266]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.7274
1   LogisticRegression(max_iter=5000)      1.000

mean test accuracy: [0.5088 0.6762 0.5072 0.6994 0.7008 0.7218 0.7272 0.726  0.7268 0.727
 0.7268 0.7268 0.7266 0.7264 0.7264 0.7264 0.7268 0.7264 0.6762 0.6994
 0.7218 0.726  0.727  0.7266 0.7264 0.7264 0.7264 0.7264 0.7264]
mean test roc auo ovr: [0.5        0.76828641 0.5        0.79064811 0.79103676 0.81185763
 0.81584111 0.8168264  0.81721217 0.81725938 0.81730262 0.81730982
 0.81732742 0.81732342 0.81732582 0.81732022 0.81732342 0.81732502
 0.76828881 0.7906465  0.81185523 0.81682961 0.81725938 0.81730662
 0.81732182 0.81732502 0.81732502 0.81732502 0.81731942]
mean test f1 micro: [0.5088 0.6762 0.5072 0.6994 0.7008 0.7218 0.7272 0.726  0.7268 0.727
 0.7268 0.7268 0.7266 0.7264 0.7264 0.7264 0.7268 0.7264 0.6762 0.6994
 0.7218 0.726  0.727  0.7266 0.7264 0.7264 0.7264 0.7264 0.7264]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l1   saga     0.7272
1   LogisticRegression(max_iter=5000)      1.000

mean test accuracy: [0.4952 0.6832 0.5028 0.7    0.703  0.7246 0.7278 0.7298 0.7272 0.7276
 0.7272 0.7272 0.7272 0.7272 0.7272 0.7272 0.7272 0.7272 0.6832 0.7
 0.7246 0.7298 0.7276 0.7272 0.7272 0.7272 0.7272 0.7272 0.7272]
mean test roc auo ovr: [0.5        0.76598321 0.5        0.78786395 0.78641419 0.8080147
 0.81186244 0.8128324  0.81314685 0.81319246 0.81321327 0.81322286
 0.81323727 0.81323647 0.81323887 0.81323727 0.81323647 0.81323007
 0.76598401 0.78786475 0.80801631 0.8128308  0.81319326 0.81322366
 0.81323247 0.81323247 0.81323327 0.81323327 0.81323167]
mean test f1 micro: [0.4952 0.6832 0.5028 0.7    0.703  0.7246 0.7278 0.7298 0.7272 0.7276
 0.7272 0.7272 0.7272 0.7272 0.7272 0.7272 0.7272 0.7272 0.6832 0.7
 0.7246 0.7298 0.7276 0.7272 0.7272 0.7272 0.7272 0.7272 0.7272]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l2   saga     0.7298
1   LogisticRegression(max_iter=5000)      0.1000    

mean test accuracy: [0.4918 0.68   0.4894 0.6934 0.6998 0.7262 0.7326 0.7348 0.7344 0.7344
 0.7344 0.7344 0.7346 0.7346 0.7346 0.7346 0.7346 0.7346 0.68   0.6934
 0.7262 0.7348 0.7344 0.7344 0.7346 0.7346 0.7346 0.7346 0.7346]
mean test roc auo ovr: [0.5        0.76775631 0.5        0.79075511 0.78643661 0.81513729
 0.82007771 0.82075841 0.82123977 0.82127902 0.82127902 0.82127981
 0.82129183 0.82128542 0.82129183 0.82128702 0.82128622 0.82128542
 0.76775551 0.7907503  0.81514049 0.8207576  0.82128222 0.82127501
 0.82128702 0.82128622 0.82128702 0.82128702 0.82128862]
mean test f1 micro: [0.4918 0.68   0.4894 0.6934 0.6998 0.7262 0.7326 0.7348 0.7344 0.7344
 0.7344 0.7344 0.7346 0.7346 0.7346 0.7346 0.7346 0.7346 0.68   0.6934
 0.7262 0.7348 0.7344 0.7344 0.7346 0.7346 0.7346 0.7346 0.7346]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l2  lbfgs     0.7348
1   LogisticRegression(max_iter=5000)      0.1

mean test accuracy: [0.4918 0.68   0.507  0.6934 0.6998 0.7262 0.7328 0.7348 0.7344 0.7344
 0.7344 0.7344 0.7346 0.7346 0.7346 0.7346 0.7346 0.7346 0.68   0.6934
 0.7262 0.7348 0.7344 0.7344 0.7346 0.7346 0.7346 0.7346 0.7346]
mean test roc auo ovr: [0.5        0.76776112 0.5        0.79075351 0.78643821 0.81513729
 0.82007851 0.8207496  0.82124137 0.82128622 0.82127902 0.82127821
 0.82128542 0.82129102 0.82128863 0.82128702 0.82128702 0.82128863
 0.76775551 0.7907503  0.81514049 0.8207576  0.82128222 0.82127501
 0.82128702 0.82128622 0.82128702 0.82128702 0.82129022]
mean test f1 micro: [0.4918 0.68   0.507  0.6934 0.6998 0.7262 0.7328 0.7348 0.7344 0.7344
 0.7344 0.7344 0.7346 0.7346 0.7346 0.7346 0.7346 0.7346 0.68   0.6934
 0.7262 0.7348 0.7344 0.7344 0.7346 0.7346 0.7346 0.7346 0.7346]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.1000      l2  lbfgs     0.7348
1   LogisticRegression(max_iter=5000)      0.1

In [42]:
letterp2_logreg_list

[array([0.4872, 0.691 , 0.4896, 0.7034, 0.6994, 0.7272, 0.7366, 0.7366,
        0.7376, 0.7378, 0.7382, 0.7382, 0.7384, 0.7384, 0.7384, 0.7384,
        0.7384, 0.7384, 0.691 , 0.7034, 0.727 , 0.7366, 0.7378, 0.7382,
        0.7384, 0.7384, 0.7384, 0.7384, 0.7384]),
 array([0.4996, 0.691 , 0.492 , 0.7034, 0.6994, 0.7274, 0.7366, 0.7366,
        0.7376, 0.738 , 0.7382, 0.7382, 0.7384, 0.7384, 0.7384, 0.7384,
        0.7384, 0.7384, 0.691 , 0.7034, 0.727 , 0.7366, 0.7378, 0.7382,
        0.7384, 0.7384, 0.7384, 0.7384, 0.7384]),
 array([0.4952, 0.691 , 0.4972, 0.7034, 0.6994, 0.727 , 0.7366, 0.7366,
        0.7376, 0.738 , 0.7382, 0.7384, 0.7384, 0.7384, 0.7384, 0.7384,
        0.7384, 0.7384, 0.691 , 0.7034, 0.727 , 0.7366, 0.7378, 0.7382,
        0.7384, 0.7384, 0.7384, 0.7384, 0.7384]),
 array([0.502 , 0.691 , 0.492 , 0.7034, 0.6994, 0.7272, 0.7366, 0.7366,
        0.7376, 0.7378, 0.7382, 0.7384, 0.7384, 0.7384, 0.7384, 0.7384,
        0.7384, 0.7384, 0.691 , 0.7034, 0.727 , 0.7366, 0.

In [43]:
letterp2_logreg_grid.cv_results_

{'mean_fit_time': array([0.00619969, 0.02059898, 0.01059923, 0.02139888, 0.02519865,
        0.0225976 , 0.02579813, 0.0229991 , 0.02639971, 0.02339883,
        0.02579937, 0.02259927, 0.02519903, 0.02439976, 0.02559891,
        0.02299929, 0.0243876 , 0.02239909, 0.01059976, 0.0105999 ,
        0.01359954, 0.01679969, 0.01679935, 0.0169991 , 0.01679907,
        0.01679997, 0.01700315, 0.01679959, 0.02299876]),
 'std_fit_time': array([3.99899493e-04, 1.20015151e-03, 1.20015147e-03, 1.01993574e-03,
        1.72036810e-03, 7.97833545e-04, 7.49361199e-04, 1.09527868e-03,
        1.62364557e-03, 4.89531705e-04, 1.16558281e-03, 2.33236711e-03,
        1.16587691e-03, 1.62352208e-03, 1.35631146e-03, 1.09532232e-03,
        7.98724288e-04, 1.20023094e-03, 4.89804209e-04, 4.89920917e-04,
        4.90193483e-04, 3.99947177e-04, 4.00257253e-04, 1.16800773e-07,
        4.00114102e-04, 3.99971094e-04, 6.37016834e-04, 3.99780416e-04,
        2.00097568e-03]),
 'mean_score_time': array([0.00519972, 

In [44]:
letterp2_logreg_grid.cv_results_['rank_test_accuracy']

array([28, 26, 29, 24, 23, 21, 20,  1, 14, 14, 17, 17,  3,  3,  3,  3,  3,
        3, 26, 24, 21,  1, 14, 17,  3,  3,  3,  3,  3])

In [45]:
letterp2_logreg_grid.cv_results_['params'][ np.argmin(letterp2_logreg_grid.cv_results_['rank_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=5000),
 'classifier__C': 0.1,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [46]:
letterp2_logreg_grid.cv_results_['params'][ np.argmin(letterp2_logreg_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'classifier': LogisticRegression(max_iter=5000),
 'classifier__C': 1000.0,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [47]:
letterp2_logreg_results = pd.DataFrame(letterp2_logreg_grid.cv_results_['params'])
letterp2_logreg_results['score_acc'] = letterp2_logreg_grid.cv_results_['mean_test_accuracy']
letterp2_logreg_cols = letterp2_logreg_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
letterp2_logreg_results.columns = letterp2_logreg_cols
letterp2_logreg_results = letterp2_logreg_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
letterp2_logreg_results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=5000),0.1000,l2,lbfgs,0.7348
1,LogisticRegression(max_iter=5000),0.1000,l2,saga,0.7348
2,LogisticRegression(max_iter=5000),1000.0000,l1,saga,0.7346
3,LogisticRegression(max_iter=5000),100.0000,l1,saga,0.7346
4,LogisticRegression(max_iter=5000),NaN,none,lbfgs,0.7346
5,LogisticRegression(max_iter=5000),10000.0000,l2,lbfgs,0.7346
6,LogisticRegression(max_iter=5000),1000.0000,l2,lbfgs,0.7346
7,LogisticRegression(max_iter=5000),100.0000,l2,lbfgs,0.7346
8,LogisticRegression(max_iter=5000),10000.0000,l2,saga,0.7346
9,LogisticRegression(max_iter=5000),10000.0000,l1,saga,0.7346


In [48]:
X_train, X_test, y_train, y_test = train_test_split(letterp2_2, letterp2_data['lettr capital letter'], train_size=5000)
sc = StandardScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test = sc.transform(X_test)
letterp2_logreg_best = linear_model.LogisticRegression(penalty='l1', C=1000, solver='saga', max_iter=5000)
letterp2_logreg_best.fit(X_train, y_train)
letterp2_pred_logreg = letterp2_logreg_best.predict(X_test)
print('Logistic Regression:', classification_report(y_test, letterp2_pred_logreg))

Logistic Regression:               precision    recall  f1-score   support

           0       0.74      0.72      0.73      7553
           1       0.72      0.75      0.73      7447

    accuracy                           0.73     15000
   macro avg       0.73      0.73      0.73     15000
weighted avg       0.73      0.73      0.73     15000



In [33]:
# KNN
for i in range(5):
    print('trial', i+1)
    letterp2_knn_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(letterp2_2, letterp2_data['lettr capital letter'], train_size=5000)
        sc = StandardScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = sc.transform(X_test)

        for k in range(5):
            letterp2_knn = KNeighborsClassifier(weights='distance')
            k_list = [1, 5 , 9 , 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 65, 69, 73, 77, 81, 85, 89, 93, 97, 101]
            letterp2_knn_grid = GridSearchCV(letterp2_knn, {'n_neighbors':k_list}, cv=KFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            letterp2_knn_grid.fit(X_train, y_train)
            print('mean test accuracy:', letterp2_knn_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', letterp2_knn_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', letterp2_knn_grid.cv_results_['mean_test_f1_micro'])
            letterp2_knn_results = pd.DataFrame(letterp2_knn_grid.cv_results_['params'])
            letterp2_knn_results['score_acc'] = letterp2_knn_grid.cv_results_['mean_test_accuracy']
            letterp2_knn_list.append(letterp2_knn_grid.cv_results_['mean_test_accuracy'])
            letterp2_knn_cols = letterp2_knn_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            letterp2_knn_results.columns = letterp2_knn_cols
            letterp2_knn_results = letterp2_knn_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(letterp2_knn_results)
            
            

trial 1
mean test accuracy: [0.9382 0.9316 0.9264 0.9182 0.9128 0.9056 0.8992 0.8928 0.8852 0.8832
 0.8792 0.8752 0.8708 0.8678 0.8682 0.867  0.8636 0.8614 0.8576 0.8534
 0.8482 0.8474 0.8474 0.8448 0.8458 0.8436]
mean test roc auo ovr: [0.93811065 0.98519499 0.98434391 0.98087431 0.97716831 0.97419067
 0.97167406 0.96839948 0.96593903 0.96372897 0.9617337  0.95933168
 0.9575416  0.95537343 0.95392435 0.95221166 0.95042936 0.94880878
 0.94695591 0.94465797 0.94298227 0.94202038 0.9402877  0.93883537
 0.93762243 0.93673528]
mean test f1 micro: [0.9382 0.9316 0.9264 0.9182 0.9128 0.9056 0.8992 0.8928 0.8852 0.8832
 0.8792 0.8752 0.8708 0.8678 0.8682 0.867  0.8636 0.8614 0.8576 0.8534
 0.8482 0.8474 0.8474 0.8448 0.8458 0.8436]
    n_neighbors  score_acc
0             1     0.9382
1             5     0.9316
2             9     0.9264
3            13     0.9182
4            17     0.9128
5            21     0.9056
6            25     0.8992
7            29     0.8928
8            33     0.

mean test accuracy: [0.9446 0.9322 0.9246 0.916  0.9102 0.9062 0.9014 0.896  0.8902 0.8854
 0.8812 0.8766 0.8748 0.8686 0.8682 0.866  0.8652 0.8642 0.8592 0.8576
 0.8584 0.8564 0.8536 0.8528 0.8502 0.8478]
mean test roc auo ovr: [0.9444941  0.98338514 0.98278004 0.979497   0.97681601 0.97348576
 0.97032728 0.96755496 0.96503787 0.96188977 0.95931442 0.95740017
 0.95552737 0.95361726 0.95161029 0.95023117 0.94861707 0.94633043
 0.94463091 0.94343416 0.94213256 0.94040324 0.93924493 0.93800011
 0.93663422 0.9354141 ]
mean test f1 micro: [0.9446 0.9322 0.9246 0.916  0.9102 0.9062 0.9014 0.896  0.8902 0.8854
 0.8812 0.8766 0.8748 0.8686 0.8682 0.866  0.8652 0.8642 0.8592 0.8576
 0.8584 0.8564 0.8536 0.8528 0.8502 0.8478]
    n_neighbors  score_acc
0             1     0.9446
1             5     0.9322
2             9     0.9246
3            13     0.9160
4            17     0.9102
5            21     0.9062
6            25     0.9014
7            29     0.8960
8            33     0.8902
9  

mean test accuracy: [0.9506 0.9414 0.9372 0.9318 0.9266 0.9182 0.9166 0.9128 0.9058 0.9004
 0.8964 0.8932 0.8896 0.887  0.886  0.8824 0.879  0.8764 0.8746 0.873
 0.8712 0.87   0.8688 0.866  0.8664 0.8656]
mean test roc auo ovr: [0.95058558 0.9875381  0.98672881 0.98427963 0.98137108 0.97867611
 0.97615807 0.97425204 0.97169253 0.9689695  0.96682658 0.96489487
 0.96301968 0.96090099 0.95859396 0.95676955 0.95516528 0.9533185
 0.95171972 0.9501467  0.94910659 0.94762425 0.94598342 0.94462023
 0.94347048 0.94190834]
mean test f1 micro: [0.9506 0.9414 0.9372 0.9318 0.9266 0.9182 0.9166 0.9128 0.9058 0.9004
 0.8964 0.8932 0.8896 0.887  0.886  0.8824 0.879  0.8764 0.8746 0.873
 0.8712 0.87   0.8688 0.866  0.8664 0.8656]
    n_neighbors  score_acc
0             1     0.9506
1             5     0.9414
2             9     0.9372
3            13     0.9318
4            17     0.9266
5            21     0.9182
6            25     0.9166
7            29     0.9128
8            33     0.9058
9     

mean test accuracy: [0.9462 0.933  0.9276 0.9204 0.9116 0.9066 0.9026 0.8956 0.8896 0.8832
 0.8804 0.876  0.8728 0.8696 0.8708 0.867  0.8648 0.8614 0.8594 0.8596
 0.8556 0.854  0.8536 0.8542 0.8518 0.8494]
mean test roc auo ovr: [0.9462822  0.98638367 0.98475752 0.98170566 0.97812591 0.97445296
 0.97158707 0.96858301 0.96642548 0.96364021 0.96109612 0.95867369
 0.95689877 0.95518152 0.95398549 0.95252152 0.95074718 0.94873247
 0.94722325 0.94534734 0.94369558 0.94183012 0.94013731 0.93869024
 0.93738396 0.93586308]
mean test f1 micro: [0.9462 0.933  0.9276 0.9204 0.9116 0.9066 0.9026 0.8956 0.8896 0.8832
 0.8804 0.876  0.8728 0.8696 0.8708 0.867  0.8648 0.8614 0.8594 0.8596
 0.8556 0.854  0.8536 0.8542 0.8518 0.8494]
    n_neighbors  score_acc
0             1     0.9462
1             5     0.9330
2             9     0.9276
3            13     0.9204
4            17     0.9116
5            21     0.9066
6            25     0.9026
7            29     0.8956
8            33     0.8896
9  

mean test accuracy: [0.9516 0.937  0.9288 0.9204 0.9166 0.9074 0.9054 0.899  0.8962 0.8912
 0.8876 0.8852 0.8806 0.8754 0.8752 0.8728 0.8728 0.8706 0.8662 0.8644
 0.8614 0.8606 0.8576 0.8552 0.8526 0.8518]
mean test roc auo ovr: [0.95170107 0.9875036  0.98492579 0.98153536 0.9782662  0.97524579
 0.97258616 0.96991746 0.9673716  0.96547804 0.96329933 0.9611664
 0.95939368 0.95726641 0.95558089 0.95413543 0.95296758 0.95121492
 0.94931329 0.94773663 0.94639429 0.94514257 0.94365215 0.9422011
 0.94087084 0.93982399]
mean test f1 micro: [0.9516 0.937  0.9288 0.9204 0.9166 0.9074 0.9054 0.899  0.8962 0.8912
 0.8876 0.8852 0.8806 0.8754 0.8752 0.8728 0.8728 0.8706 0.8662 0.8644
 0.8614 0.8606 0.8576 0.8552 0.8526 0.8518]
    n_neighbors  score_acc
0             1     0.9516
1             5     0.9370
2             9     0.9288
3            13     0.9204
4            17     0.9166
5            21     0.9074
6            25     0.9054
7            29     0.8990
8            33     0.8962
9    

...

mean test accuracy: [0.9426 0.9318 0.9248 0.9168 0.9088 0.9072 0.9018 0.8962 0.8914 0.8872
 0.881  0.8796 0.879  0.8762 0.8714 0.8708 0.8678 0.8626 0.8604 0.8588
 0.8586 0.8586 0.8578 0.8556 0.8548 0.8532]
mean test roc auo ovr: [0.94260945 0.98454747 0.98255256 0.98016647 0.97642245 0.97433537
 0.97124631 0.96864745 0.96568665 0.96373658 0.96165253 0.96013382
 0.95826269 0.95658699 0.95469666 0.95258165 0.95108039 0.94915339
 0.94760029 0.94598367 0.94461935 0.94303794 0.94163836 0.9403127
 0.93853138 0.93751527]
mean test f1 micro: [0.9426 0.9318 0.9248 0.9168 0.9088 0.9072 0.9018 0.8962 0.8914 0.8872
 0.881  0.8796 0.879  0.8762 0.8714 0.8708 0.8678 0.8626 0.8604 0.8588
 0.8586 0.8586 0.8578 0.8556 0.8548 0.8532]
    n_neighbors  score_acc
0             1     0.9426
1             5     0.9318
2             9     0.9248
3            13     0.9168
4            17     0.9088
5            21     0.9072
6            25     0.9018
7            29     0.8962
8            33     0.8914
9   

mean test accuracy: [0.9464 0.9364 0.9268 0.9244 0.918  0.9092 0.9054 0.8994 0.8974 0.893
 0.891  0.8904 0.8864 0.883  0.8772 0.8762 0.8764 0.8748 0.8696 0.8704
 0.8684 0.866  0.8642 0.8636 0.8606 0.8582]
mean test roc auo ovr: [0.94642947 0.98660657 0.98484802 0.98211296 0.97917329 0.97555289
 0.972904   0.9705476  0.96813966 0.96615656 0.96452286 0.96291046
 0.96101011 0.95926189 0.95780047 0.95586728 0.954834   0.95349005
 0.95189227 0.95075879 0.94943161 0.94838127 0.94679961 0.94552305
 0.94407805 0.94236612]
mean test f1 micro: [0.9464 0.9364 0.9268 0.9244 0.918  0.9092 0.9054 0.8994 0.8974 0.893
 0.891  0.8904 0.8864 0.883  0.8772 0.8762 0.8764 0.8748 0.8696 0.8704
 0.8684 0.866  0.8642 0.8636 0.8606 0.8582]
    n_neighbors  score_acc
0             1     0.9464
1             5     0.9364
2             9     0.9268
3            13     0.9244
4            17     0.9180
5            21     0.9092
6            25     0.9054
7            29     0.8994
8            33     0.8974
9    

mean test accuracy: [0.9438 0.9374 0.931  0.9204 0.9132 0.909  0.9046 0.8996 0.8958 0.8908
 0.886  0.8824 0.8778 0.8718 0.8706 0.8694 0.8624 0.86   0.8566 0.8538
 0.8516 0.8496 0.8478 0.8444 0.8428 0.8402]
mean test roc auo ovr: [0.94376673 0.98644933 0.984348   0.98094738 0.97759166 0.9742781
 0.97166158 0.96948319 0.96692539 0.96425751 0.96180708 0.95907038
 0.95696913 0.95492869 0.95304571 0.95102373 0.94854379 0.94709953
 0.94576903 0.94441522 0.9425501  0.94097717 0.9396594  0.93832803
 0.93697852 0.93588729]
mean test f1 micro: [0.9438 0.9374 0.931  0.9204 0.9132 0.909  0.9046 0.8996 0.8958 0.8908
 0.886  0.8824 0.8778 0.8718 0.8706 0.8694 0.8624 0.86   0.8566 0.8538
 0.8516 0.8496 0.8478 0.8444 0.8428 0.8402]
    n_neighbors  score_acc
0             1     0.9438
1             5     0.9374
2             9     0.9310
3            13     0.9204
4            17     0.9132
5            21     0.9090
6            25     0.9046
7            29     0.8996
8            33     0.8958
9   

mean test accuracy: [0.9464 0.938  0.9344 0.9272 0.9188 0.9094 0.9056 0.8986 0.8928 0.8904
 0.8866 0.8814 0.879  0.8774 0.8732 0.8686 0.8682 0.8656 0.8642 0.8596
 0.8572 0.8534 0.8502 0.8494 0.8476 0.846 ]
mean test roc auo ovr: [0.9464408  0.98758466 0.98684706 0.98283279 0.97912001 0.97579877
 0.97278091 0.96997022 0.96710613 0.9647546  0.96291711 0.9609276
 0.95889    0.95659553 0.95446522 0.95205139 0.9497386  0.94795392
 0.94615394 0.94421872 0.9425244  0.94102947 0.93952403 0.9380739
 0.93700617 0.93527027]
mean test f1 micro: [0.9464 0.938  0.9344 0.9272 0.9188 0.9094 0.9056 0.8986 0.8928 0.8904
 0.8866 0.8814 0.879  0.8774 0.8732 0.8686 0.8682 0.8656 0.8642 0.8596
 0.8572 0.8534 0.8502 0.8494 0.8476 0.846 ]
    n_neighbors  score_acc
0             1     0.9464
1             5     0.9380
2             9     0.9344
3            13     0.9272
4            17     0.9188
5            21     0.9094
6            25     0.9056
7            29     0.8986
8            33     0.8928
9    

mean test accuracy: [0.9522 0.9412 0.9358 0.9276 0.9202 0.9174 0.9092 0.9038 0.8962 0.8928
 0.8888 0.8846 0.8816 0.8778 0.875  0.8718 0.8674 0.8652 0.8638 0.8614
 0.8604 0.8606 0.8578 0.857  0.8566 0.8526]
mean test roc auo ovr: [0.95225469 0.98846706 0.98698277 0.98362548 0.98021454 0.97736912
 0.97435378 0.97165647 0.96893468 0.96656504 0.9643044  0.96190139
 0.95945819 0.95740962 0.95525763 0.95294104 0.95110709 0.9495074
 0.94796708 0.94699167 0.94578824 0.94452391 0.94319309 0.94143401
 0.94031207 0.93900524]
mean test f1 micro: [0.9522 0.9412 0.9358 0.9276 0.9202 0.9174 0.9092 0.9038 0.8962 0.8928
 0.8888 0.8846 0.8816 0.8778 0.875  0.8718 0.8674 0.8652 0.8638 0.8614
 0.8604 0.8606 0.8578 0.857  0.8566 0.8526]
    n_neighbors  score_acc
0             1     0.9522
1             5     0.9412
2             9     0.9358
3            13     0.9276
4            17     0.9202
5            21     0.9174
6            25     0.9092
7            29     0.9038
8            33     0.8962
9   

In [34]:
letterp2_knn_list

[array([0.9464, 0.9364, 0.9268, 0.9244, 0.918 , 0.9092, 0.9054, 0.8994,
        0.8974, 0.893 , 0.891 , 0.8904, 0.8864, 0.883 , 0.8772, 0.8762,
        0.8764, 0.8748, 0.8696, 0.8704, 0.8684, 0.866 , 0.8642, 0.8636,
        0.8606, 0.8582]),
 array([0.9464, 0.9364, 0.9268, 0.9244, 0.918 , 0.9092, 0.9054, 0.8994,
        0.8974, 0.893 , 0.891 , 0.8904, 0.8864, 0.883 , 0.8772, 0.8762,
        0.8764, 0.8748, 0.8696, 0.8704, 0.8684, 0.866 , 0.8642, 0.8636,
        0.8606, 0.8582]),
 array([0.9464, 0.9364, 0.9268, 0.9244, 0.918 , 0.9092, 0.9054, 0.8994,
        0.8974, 0.893 , 0.891 , 0.8904, 0.8864, 0.883 , 0.8772, 0.8762,
        0.8764, 0.8748, 0.8696, 0.8704, 0.8684, 0.866 , 0.8642, 0.8636,
        0.8606, 0.8582]),
 array([0.9464, 0.9364, 0.9268, 0.9244, 0.918 , 0.9092, 0.9054, 0.8994,
        0.8974, 0.893 , 0.891 , 0.8904, 0.8864, 0.883 , 0.8772, 0.8762,
        0.8764, 0.8748, 0.8696, 0.8704, 0.8684, 0.866 , 0.8642, 0.8636,
        0.8606, 0.8582]),
 array([0.9464, 0.9364, 0.9268, 

In [35]:
letterp2_knn_grid.cv_results_

{'mean_fit_time': array([0.0024004 , 0.00279994, 0.00259995, 0.00240016, 0.00260005,
        0.00240006, 0.00219994, 0.00240026, 0.00219984, 0.0026001 ,
        0.0019999 , 0.00220027, 0.0026    , 0.00220013, 0.00240006,
        0.00240006, 0.00219994, 0.00219984, 0.0026    , 0.00240021,
        0.00280032, 0.00279999, 0.00240002, 0.00259991, 0.00219998,
        0.00259976]),
 'std_fit_time': array([4.90758084e-04, 4.00042829e-04, 4.89979288e-04, 4.89979567e-04,
        4.90057581e-04, 4.89959905e-04, 4.00161772e-04, 4.89804070e-04,
        4.00090228e-04, 4.90096112e-04, 2.78041453e-07, 4.00233365e-04,
        4.89920871e-04, 3.99947234e-04, 4.89862441e-04, 4.90154611e-04,
        4.00161828e-04, 3.99971378e-04, 4.90310284e-04, 4.90037718e-04,
        4.00114130e-04, 4.00781702e-04, 4.89901382e-04, 4.89745733e-04,
        4.00018706e-04, 4.89921172e-04]),
 'mean_score_time': array([0.12779546, 0.178794  , 0.17939439, 0.18079419, 0.18179398,
        0.18159404, 0.18419385, 0.1875937 , 

In [36]:
letterp2_knn_grid.cv_results_['rank_test_accuracy']

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 22, 21, 23, 24, 25, 26])

In [37]:
letterp2_knn_grid.cv_results_['params'][ np.argmin(letterp2_knn_grid.cv_results_['rank_test_accuracy'])]

{'n_neighbors': 1}

In [38]:
letterp2_knn_grid.cv_results_['params'][ np.argmin(letterp2_knn_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'n_neighbors': 5}

In [39]:
letterp2_knn_results = pd.DataFrame(letterp2_knn_grid.cv_results_['params'])
letterp2_knn_results['score_acc'] = letterp2_knn_grid.cv_results_['mean_test_accuracy']
letterp2_knn_cols = letterp2_knn_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
letterp2_knn_results.columns = letterp2_knn_cols
letterp2_knn_results = letterp2_knn_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
letterp2_knn_results

,n_neighbors,score_acc
0,1,0.9522
1,5,0.9412
2,9,0.9358
3,13,0.9276
4,17,0.9202
5,21,0.9174
6,25,0.9092
7,29,0.9038
8,33,0.8962
9,37,0.8928


In [40]:
X_train, X_test, y_train, y_test = train_test_split(letterp2_2, letterp2_data['lettr capital letter'], train_size=5000)
sc = StandardScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test = sc.transform(X_test)
letterp2_knn_best = KNeighborsClassifier(weights='distance', n_neighbors=5)
letterp2_knn_best.fit(X_train, y_train)
letterp2_pred_knn = letterp2_knn_best.predict(X_test)
print('KNN:', classification_report(y_test, letterp2_pred_knn))


KNN:               precision    recall  f1-score   support

           0       0.95      0.95      0.95      7519
           1       0.95      0.95      0.95      7481

    accuracy                           0.95     15000
   macro avg       0.95      0.95      0.95     15000
weighted avg       0.95      0.95      0.95     15000

